In [1]:
import pandas as pd
import numpy as np

from utils.personal_utils import *

import mlflow
from tsfeatures import tsfeatures
import os

from utils.compression_algos import *

In [2]:
# Load Different Datasets to see how they are:

all_datasets = [
                "/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TEST.ts", 
                "/home/sim/Desktop/TS Extrinsic Regression/data/BeijingPM25Quality_TEST.ts",
                "/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts",
                "/home/sim/Desktop/TS Extrinsic Regression/data/NewsTitleSentiment_TEST.ts",
                "/home/sim/Desktop/TS Extrinsic Regression/data/Covid3Month_TEST.ts",
                





                "/home/sim/Desktop/TS Extrinsic Regression/data/BenzeneConcentration_TEST.ts",
                "/home/sim/Desktop/TS Extrinsic Regression/data/FloodModeling1_TEST.ts",
                "/home/sim/Desktop/TS Extrinsic Regression/data/HouseholdPowerConsumption1_TEST.ts",
                ]
loaded_datasets = []

for dataset_path in all_datasets:
    loaded_datasets.append(load_dataset(dataset_path))


    print(os.path.basename(dataset_path).split('_')[0], " ", loaded_datasets[-1].shape)
                





66it [00:01, 56.39it/s] 
100%|██████████| 42/42 [00:00<00:00, 505.52it/s]


AppliancesEnergy   (42, 144, 24)


5072it [00:11, 439.33it/s]
100%|██████████| 5048/5048 [00:05<00:00, 995.09it/s] 


BeijingPM25Quality   (5048, 24, 9)


1351it [00:01, 801.51it/s]
100%|██████████| 1328/1328 [00:00<00:00, 1431.10it/s]


...

BenzeneConcentration   (5163, 240, 8)


216it [00:00, 8283.23it/s]
100%|██████████| 202/202 [00:00<00:00, 2521.68it/s]


FloodModeling1   (202, 266, 1)


706it [00:30, 23.18it/s]
100%|██████████| 686/686 [00:00<00:00, 1315.07it/s]

HouseholdPowerConsumption1   (686, 1440, 5)


In [2]:
path = "/home/sim/Desktop/TS Extrinsic Regression/data/IEEEPPG_TEST.ts"
a = load_dataset(path)

1351it [00:01, 691.05it/s]
100%|██████████| 1328/1328 [00:01<00:00, 1310.86it/s]


In [23]:
#Test if for even input decompressed array can have bigger size than normal!

for i in range(0,2002,1):

    long_array = np.random.rand(i)

    if dwt_compress(long_array, 0.1, False).shape[0] - i not in [0,1,2]:
        print(i, dwt_compress(long_array, 0.1, False).shape[0] - i)



# Learned:(with max level and on input signal of size 2000)
# The decompressed signal is in ever case max 1 bigger and only if it is uneven!
# The coefficients and with max_level can be up to 2,3,4 bigger. Also for signals of length < 2000. On it's way to 2000 it can even get up to 7.


57 3
65 3
73 3
81 3
89 3
97 3
105 3
113 4
114 3
115 3
117 3
121 3
129 4
130 3
131 3
133 3
137 3
145 4
146 3
147 3
149 3
153 3
161 4
162 3
163 3
165 3
169 3
177 4
178 3
179 3
181 3
185 3
193 4
194 3
195 3
197 3
201 3
209 4
210 3
211 3
213 3
217 3
225 5
226 4
227 4
228 3
229 4
230 3
231 3
233 4
234 3
235 3
237 3
241 4
242 3
243 3
245 3
249 3
257 5
258 4
259 4
260 3
261 4
262 3
263 3
265 4
266 3
267 3
269 3
273 4
274 3
275 3
277 3
281 3
289 5
290 4
291 4
292 3
293 4
294 3
295 3
297 4
298 3
299 3
301 3
305 4
306 3
307 3
309 3
313 3
321 5
322 4
323 4
324 3
325 4
326 3
327 3
329 4
330 3
331 3
333 3
337 4
338 3
339 3
341 3
345 3
353 5
354 4
355 4
356 3
357 4
358 3
359 3
361 4
362 3
363 3
365 3
369 4
370 3
371 3
373 3
377 3
385 5
386 4
387 4
388 3
389 4
390 3
391 3
393 4
394 3
395 3
397 3
401 4
402 3
403 3
405 3
409 3
417 5
418 4
419 4
420 3
421 4
422 3
423 3
425 4
426 3
427 3
429 3
433 4
434 3
435 3
437 3
441 3
449 6
450 5
451 5
452 4
453 5
454 4
455 4
456 3
457 5
458 4
459 4
460 3
461 4
462 

In [4]:
# Test dimension flattening for compresssion
import numpy as np

# Define the array dimensions
num_dp = 6
len_ts = 3
dim = 4

# Create an example array with shape (6, 3, 4)
data = np.array([
    [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12]
    ],
    [
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]
    ],
    [
        [25, 26, 27, 28],
        [29, 30, 31, 32],
        [33, 34, 35, 36]
    ],
    [
        [37, 38, 39, 40],
        [41, 42, 43, 44],
        [45, 46, 47, 48]
    ],
    [
        [49, 50, 51, 52],
        [53, 54, 55, 56],
        [57, 58, 59, 60]
    ],
    [
        [61, 62, 63, 64],
        [65, 66, 67, 68],
        [69, 70, 71, 72]
    ]
])

print("Original array:")
print(data)
print("Original shape:", data.shape)
# Output: (6, 3, 4)

# Reshape the array to (num_dp * len_ts, dim)
flattened_data = data.reshape(-1, dim)

print("\nFlattened array:")
print(flattened_data)
print("New shape:", flattened_data.shape)
# Output: (18, 4)


Original array:
[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]]

 [[13 14 15 16]
  [17 18 19 20]
  [21 22 23 24]]

 [[25 26 27 28]
  [29 30 31 32]
  [33 34 35 36]]

 [[37 38 39 40]
  [41 42 43 44]
  [45 46 47 48]]

 [[49 50 51 52]
  [53 54 55 56]
  [57 58 59 60]]

 [[61 62 63 64]
  [65 66 67 68]
  [69 70 71 72]]]
Original shape: (6, 3, 4)

Flattened array:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]
 [25 26 27 28]
 [29 30 31 32]
 [33 34 35 36]
 [37 38 39 40]
 [41 42 43 44]
 [45 46 47 48]
 [49 50 51 52]
 [53 54 55 56]
 [57 58 59 60]
 [61 62 63 64]
 [65 66 67 68]
 [69 70 71 72]]
New shape: (18, 4)


In [5]:
import numpy as np
from numpy.polynomial.chebyshev import chebfit, chebval


# Chebyshev Transoform Compression Code -WIP
def cpt_compress(signal, dropout_ratio, andDecompress:bool):
    
    # Later: Try with fitting into Interval -1,1

    # Evaluation points
    t = np.arange(signal.size)

    # Get the coefficients. deg is the longest that is possible. Shouldn't take too long to calculate.
    cpt_coeffs = chebfit(t, signal, deg=len(signal)-1)

    # Calculate the number of coefficients to zero out
    num_coeffs = int((dropout_ratio) * len(cpt_coeffs))
    
    # Sort the coefficients by value and cut off the smallest ones
    sorted_indices = np.argsort(np.abs(cpt_coeffs))
    indices_to_zero = sorted_indices[:num_coeffs]

    # Zero out selected coefficients
    cpt_coeffs[indices_to_zero] = 0

    if andDecompress == False:
        return cpt_coeffs
    else:
        decompressed_signal = chebval(t, cpt_coeffs)
        return decompressed_signal


In [6]:
cpt_compress(np.array(), 0.9, True)

TypeError: array() missing required argument 'object' (pos 0)